In [1]:
import string
import os
import random
import numpy as np
from unidecode import unidecode
from torch.utils.data import Dataset, DataLoader, BatchSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import random_split
from collections import Counter


In [ ]:
NAMES_PATH = 'data/NameClassifyingPytorch/names'
ASCII = string.ascii_letters
ASCII_COUNT = len(ASCII)
COUNTRIES = os.listdir(NAMES_PATH)
BATCH_SIZE = 64

random.seed(1711)

def char_onehot(c):
    if c not in ASCII:
        return []
    
    result = [0.0] * ASCII_COUNT    
    result[ASCII.index(c)] = 1.0
    return result

In [3]:
def ascii_name_onehot(name):
    name_onehot = []

    for c in name:
        c = char_onehot(c)
        if c:
            name_onehot.append(c)

    return name_onehot

In [4]:
# From path to a whole Dataset
class NameDataset(Dataset):
    def __init__(self, name_original, name_unicode, name_tensor, label):
        self.name_original = name_original  # Each row is a original name
        self.name_unicode = name_unicode    # Each row is a unicoded name
        self.name_tensor = name_tensor      # Each row is a name tensor
        self.label = label                  # Each row is a country label

    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        if isinstance(idx, list):
            return [self.name_original[i] for i in idx],\
                [self.name_unicode[i] for i in idx],\
                [self.name_tensor[i] for i in idx],\
                [self.label[i] for i in idx]
        
        return self.name_original[idx],\
                self.name_unicode[idx],\
                self.name_tensor[idx],\
                self.label[idx]

In [5]:
class NameRNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, nonlinearity='tanh', bias=True, batch_first=True)
        self.fcc = nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        rnn_out = rnn_out[:,-1,:] # batch_size, layer, dimensions
        rnn_logits = self.fcc(rnn_out)
        rnn_softmax = self.softmax(rnn_logits)
        return rnn_softmax

In [6]:
class NameGRU(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super().__init__()
        self.gru = nn.GRU(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bias=True, batch_first=True)
        self.fcc = nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = gru_out[:,-1,:] # batch_size, layer, dimensions
        gru_logits = self.fcc(gru_out)
        gru_softmax = self.softmax(gru_logits)
        return gru_softmax

In [7]:
class NameLSTM(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bias=True, batch_first=True)
        self.fcc = nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:,-1,:] # batch_size, layer, dimensions
        lstm_logits = self.fcc(lstm_out)
        lstm_softmax = self.softmax(lstm_logits)
        return lstm_softmax

In [8]:
def get_dataset_dict(path):
    dataset_dict = {} # name_length to dataset
    
    for filename in os.listdir(path):
        with open(path + "/" + filename, encoding='utf-8') as f:
            for name in f.readlines():
                name = name.strip()
                nameunicode = unidecode(name.replace(" ", ""))
                name_onehot = ascii_name_onehot(nameunicode)
                if name_onehot:
                    name_len = len(name_onehot)
                    if name_len not in dataset_dict:
                        dataset_dict[len(name_onehot)] = [[], [], [], []]
                    
                    dataset_dict[name_len][0].append(name)
                    dataset_dict[name_len][1].append(nameunicode)
                    dataset_dict[name_len][2].append(name_onehot)
                    dataset_dict[name_len][3].append(COUNTRIES.index(filename))

    result = {}
    for length, ds in dataset_dict.items():
        ds[2] = torch.tensor(ds[2], dtype=torch.float)
        ds[3] = torch.tensor(ds[3], dtype=torch.long)
        result[length] = NameDataset(ds[0], ds[1], ds[2], ds[3])
    
    return result
# train_set, test_set = torch.utils.data.random_split(alldata, [.85, .15], 

In [9]:
dataset_dict = get_dataset_dict(NAMES_PATH)
dataloader_dict = {i: DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True) for i, ds in dataset_dict.items()}

In [10]:
def train(model):
    epochs = 200
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    for epoch in range(epochs):
        epoch_loss = 0
        for _, dataloader in dataloader_dict.items():
            dl_loss = 0
            for dl in dataloader:
                origin_name, unicode_name, onehot_name, country_label = dl            
                x_out = model(onehot_name)
                loss = F.cross_entropy(x_out, country_label, reduction='mean')
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                dl_loss += loss.item()
            epoch_loss += dl_loss / len(dataloader)
        if epoch % 50 == 0:
            print(f"Epoch={epoch}: Loss={epoch_loss}")

In [11]:
def make_prediction(model, name):
    model.eval()
    name = unidecode(name)
    name_onehot = ascii_name_onehot(name)
    name_onehot = torch.tensor(name_onehot)
    name_onehot = name_onehot.unsqueeze(0)  # A batch of one datapoint
    output = model(name_onehot)
    predicted_idx = output.argmax(dim=1)
    return COUNTRIES[predicted_idx.item()]

In [12]:
def eval(model):
    for file_name in os.listdir(NAMES_PATH):
        pred_countries = []
        with open(NAMES_PATH + "/" + file_name, encoding='utf-8') as f:
            for line in f.readlines():
                name = line.strip().replace(" ", "")
                predicted_country = make_prediction(model, name)
                pred_countries.append(predicted_country)
        freq_dict = dict(Counter(pred_countries))
        acc = freq_dict.get(file_name, 0) / len(pred_countries) * 100
        print(f"{file_name}: {acc:.2f}%")

In [13]:
rnn_model = NameRNN(input_dim=ASCII_COUNT, output_dim=len(COUNTRIES))
train(rnn_model)
eval(rnn_model)

Epoch=0: Loss=33.568166097694856
Epoch=50: Loss=2.1459467083174477
Epoch=100: Loss=1.4859893803127218
Epoch=150: Loss=2.1872763089031353
Czech.txt: 96.72%
German.txt: 91.99%
Arabic.txt: 100.00%
Japanese.txt: 99.29%
Chinese.txt: 92.91%
Vietnamese.txt: 84.93%
Russian.txt: 99.56%
French.txt: 81.23%
Irish.txt: 80.60%
English.txt: 96.54%
Spanish.txt: 88.26%
Greek.txt: 99.51%
Italian.txt: 97.18%
Portuguese.txt: 48.65%
Scottish.txt: 21.00%
Dutch.txt: 92.93%
Korean.txt: 89.36%
Polish.txt: 92.09%


In [14]:
gru_model = NameGRU(input_dim=ASCII_COUNT, output_dim=len(COUNTRIES))
train(gru_model)
eval(gru_model)

Epoch=0: Loss=23.42127067853795
Epoch=50: Loss=1.1994503026970795
Epoch=100: Loss=0.9302142684005948
Epoch=150: Loss=0.9222299609238289
Czech.txt: 97.11%
German.txt: 89.78%
Arabic.txt: 100.00%
Japanese.txt: 99.50%
Chinese.txt: 89.18%
Vietnamese.txt: 84.93%
Russian.txt: 99.63%
French.txt: 89.89%
Irish.txt: 78.02%
English.txt: 96.26%
Spanish.txt: 84.56%
Greek.txt: 100.00%
Italian.txt: 96.47%
Portuguese.txt: 68.92%
Scottish.txt: 47.00%
Dutch.txt: 95.29%
Korean.txt: 79.79%
Polish.txt: 91.37%


In [15]:
lstm_model = NameLSTM(input_dim=ASCII_COUNT, output_dim=len(COUNTRIES))
train(lstm_model)
eval(lstm_model)

Epoch=0: Loss=35.43046084012918
Epoch=50: Loss=1.4640627551203835
Epoch=100: Loss=1.047924185571055
Epoch=150: Loss=0.9119874319611514
Czech.txt: 96.92%
German.txt: 92.82%
Arabic.txt: 100.00%
Japanese.txt: 99.50%
Chinese.txt: 90.30%
Vietnamese.txt: 86.30%
Russian.txt: 99.64%
French.txt: 86.64%
Irish.txt: 79.31%
English.txt: 96.97%
Spanish.txt: 82.89%
Greek.txt: 100.00%
Italian.txt: 97.04%
Portuguese.txt: 74.32%
Scottish.txt: 6.00%
Dutch.txt: 91.92%
Korean.txt: 79.79%
Polish.txt: 92.81%
